# connection to DB config and open

In [ ]:
import pyodbc
import pandas as pd
import glob
server = '<server>.database.windows.net'
database = '<database>'
username = '<username>'
password = '<password>'
driver= '{ODBC Driver 17 for SQL Server}'
conn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

# create graph tables

In [30]:
crsr = conn.cursor()
crsr.execute("CREATE TABLE Clubs (ID INTEGER PRIMARY KEY, Name VARCHAR(100)) AS NODE")
crsr.execute("CREATE TABLE Players (ID INTEGER PRIMARY KEY, Name VARCHAR(100)) AS NODE")
crsr.execute("CREATE TABLE Squad (year INTEGER, joined INTEGER) AS EDGE")
conn.commit()
crsr.close()

# load clubs to the node

In [17]:
clubs = pd.read_csv('../data/clubs.csv')

cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in clubs.iterrows():
     cursor.execute("INSERT INTO dbo.Clubs (ID, Name) values(?,?)", row['id'], row['name'])
conn.commit()
cursor.close()

# load unique players to Players node

In [ ]:
files = glob.glob("..\data\player*.csv")
dfs = [pd.read_csv(f) for f in files]

players = pd.concat(dfs,ignore_index=True).loc[:, ['id', 'name']].drop_duplicates()
print(players.head())

cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in players.iterrows():
     cursor.execute("INSERT INTO dbo.Players (ID, Name) values(?,?)", row['id'], row['name'])
conn.commit()
cursor.close()

# load rosters year-by-year

In [33]:
files = glob.glob("..\data\players*.csv")
dfs = [pd.read_csv(f) for f in files]

players = pd.concat(dfs,ignore_index=True)
players['joined'].replace('False', 0, inplace=True)

cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in players.iterrows():
     cursor.execute("INSERT INTO dbo.Squad values ((SELECT $node_id FROM Clubs WHERE ID = ?),(SELECT $node_id FROM Players WHERE ID = ?), ?, ?)", row['club'], row['id'], row['year'], row['joined'])
conn.commit()
cursor.close()

In [28]:
conn.close()